# A- Generate daily plan

## Daily plan JSON structure

In [2]:
plan = {
    "date": "",
    
    "start_location": "",
    "places_to_visit": [
        {
            "name": "",
            "location": "",
            "validated": False
        },
        {
            "name": "",
            "location": "",
            "validated": False
        }
    ],
    "sport_exercises": [
        {
            "exercise": "",
            "count_goal": 0,
            "count_done": 0,
        },
        {
            "exercise": "",
            "count_goal": 0,
            "count_done": 0,
        }
    ],
    "gained_coins": 0
}

## Find places to visit in the near by
## Google Places API - Nearby Search 

In [21]:
import requests
import googlemaps # Github: https://github.com/googlemaps/google-maps-services-python/tree/master
import APIs_keys as keys #API_keys.py has been added to .gitignore

In [36]:
#init python-google-maps-api client
try:
    del gmaps
except:
    pass
gmaps = googlemaps.Client(key=keys.GOOGLE_PLACES_API_KEY)

In [41]:
# for tinkering, use Aschaffenburg, DE long and lat. Later we get them from the web browser geolocation.
start_location = {
    "lat": 49.9807,
    "long": 9.1356,
}

# another way to get geometry is to use gmaps Geocode API
"""
geocode_result = gmaps.geocode('Aschaffenburg Hbf, Aschaffenburg, DE')
geocode_result[0]["geometry"]["location"]
res: 
{'lat': 49.98056, 'lng': 9.143699999999999}
"""

# distance from search center in meter (int)
search_radius = 40000 #40KM

res = gmaps.places_nearby(
    location = (start_location["lat"], start_location["long"]),
    radius = search_radius,
    type = ["cafe", "restaurant"], # full supported values: https://developers.google.com/maps/documentation/places/web-service/supported_types
    language = "en")

print(res)

{'html_attributions': [], 'next_page_token': 'AUc7tXUTYRxLqxVz3z0BqiGite4SPLqp5JUAXVh2xlqvdAJqhAFyx6q5-7K38WQr8kBtjTQILdcfv7h2nFzOYWVqRZJMMIt-27txryjV2r8UOlLPSRC21QlqRSh97dXfQUl9OcQykm3jEy3C0hE494n1UTQaQXF6j1r-Jjgc1WGefoF1X5q7NskN9i-3eCxrii76yu0VnqVmqNgHVfWRKrOvX1NSWC3XLh2_EiBwejo4Kv_kmBlTTtY--8eDAqTjtbaRuE_liTFV17n_QyOcSbhtLpcCKfs4HcjVB94U901-D0QVDYAhATyUVFolQPHBAI8aW_yiU9iNriDEI2FwDGvdzNzwmTspfWXSUbkH1zAaqkL0NAgC41ut9rj9Iz2Ux9oXcnQ5P6Ev-6VZSLkC94ZlwEu4SqQ41tfegq-JhcO0Vs45Cmvf', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 49.96127, 'lng': 9.390789999999999}, 'viewport': {'northeast': {'lat': 49.9624611802915, 'lng': 9.392339080291501}, 'southwest': {'lat': 49.9597632197085, 'lng': 9.389641119708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Spechtshaardt', 'photos

In [42]:
res["results"]

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 49.96127, 'lng': 9.390789999999999},
   'viewport': {'northeast': {'lat': 49.9624611802915,
     'lng': 9.392339080291501},
    'southwest': {'lat': 49.9597632197085, 'lng': 9.389641119708498}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'icon_background_color': '#909CE1',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
  'name': 'Hotel Spechtshaardt',
  'photos': [{'height': 837,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118166564526583613480">Hotel Restaurant Spechtshaardt</a>'],
    'photo_reference': 'AUc7tXWYewTSgnwgX0_qj4hToP-xMv5ANn7FD05kOKyOi_oWbrr7G00o-NQLvtvY5owNozgjMFKlTTo3YZmOT4ZHN2rXHJy8vI1JAVmZ_wyFlHWyHcYbj8EqcUdKQGPa6qDJ6m-MnpAnuPFi5Awm9uiffnQpFkxWCpDcnUVkjp-N-2DGv7Ek',
    'width': 1677}],
  'place_id': 'ChIJm0MT8V21okcRC6i0CbwWl8o',
  'plus_code': {'compound_code': 'X96R+G8 Rot